In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/amandawasemiller/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [3]:
#First go to NASA news
url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

soup = BeautifulSoup(browser.html)

slide = soup.find("li", {"class": "slide"})
news_title = slide.find("div", {"class": "content_title"}).text
news_body = slide.find("div", {"class": "article_teaser_body"}).text.strip()

In [4]:
news_title

"Celebrate the Perseverance Rover Landing With NASA's Student Challenge"

In [5]:
news_body

'The rover touches down on the Red Planet next month, and students are invited to join the excitement by designing, building, and landing their own Mars mission. NASA can help.'

In [6]:
# I got stuck on this one. The link in the HW did not return a clickable image. Only a searchable gallery. I realize what is shown is not what was intended.
base = "https://www.jpl.nasa.gov"
url = f"{base}/images/supercams-mars-meteorite-aboard-the-iss/" # This was the current featured image. I reviewed the office hours recording and chose to navigate to the featured gallery image.
browser.visit(url)
time.sleep(1)

browser.find_by_id("82498").click()
time.sleep(1)

soup = BeautifulSoup(browser.html)
image = soup.find("img", {"class": "82498"})

#clickable image at the end of this code
#SOURCE:https://stackoverflow.com/questions/56943210/how-to-get-src-attribute-from-image-with-python
featured_image_url = soup.findAll('img')[2]
print(featured_image_url['src'])

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.width-1024.jpg
